In [7]:
from keras.models import load_model
import numpy as np

In [4]:
class Generator () :
    
    def __init__(self, model_file_path:str, vocab_file_path : str="./../Data/Dataset/Matthieu_processed.txt", seq_len=150):

        self.__model = load_model(model_file_path)

        self.__vocab_file_path:str = vocab_file_path
        self.__vocab:dict = self.__init_vocab()

        self.__int_to_char:dict = dict((i, c) for i, c in enumerate(self.__vocab))
        self.__char_to_int:dict = dict ((c,i) for i, c in enumerate(self.__vocab))

        self.__sequence_length:int = seq_len
    
    def __init_vocab (self) -> dict :

        with open (self.__vocab_file_path,"r") as file :

            vocab = sorted(set(file.read()))

        return vocab

    def generate_sentence (self, seed : str, numbeOfCharToGenerate : int=500, save_to_file:bool=False, file_to_save:str=None) -> str :

        if len (seed) >= self.__sequence_length :

            print ("Your seed text should not exceed : " + str(self.__sequence_length))
            return None

        original_sentence = seed.lower()

        # Generate Sentence
        sentence = ""

        # Lower the input 
        input_seed = seed.lower()

        #replace "'"
        input_seed = input_seed.replace("'", " ")

        # Transform the ascii char to vector of int
        input_seed = [self.__char_to_int[char] for char in input_seed]

        # Padding Element
        for i in range (self.__sequence_length-len (input_seed)) :
            input_seed.append(0)

        # Put in LSTM format
        input_seed = np.reshape(input_seed, (1, len(input_seed), 1))

        # Squeeze to float
        input_seed = input_seed / float(len (self.__vocab))

        for _ in range (numbeOfCharToGenerate):

            prediction = self.__model.predict(input_seed, verbose=0)
            index = np.argmax(prediction)
            predicted_char = self.__int_to_char[index]
            sentence = sentence+str(predicted_char)

            # update Input Seed
            predicted_char_int:int = self.__char_to_int[predicted_char]
            predicted_char_int:float = predicted_char_int/float(len (self.__vocab))
            input_seed = np.append(input_seed, predicted_char_int)
            input_seed = np.delete(input_seed, 0) 
            input_seed = np.reshape(input_seed, (1, len(input_seed), 1))

        sentence = original_sentence + sentence

        if save_to_file == True :

            if file_to_save == None :

                print ("Did not save to file : File Path argument = None")
            else :
                with open(file_to_save, "w") as file :
                    file.write(sentence)

        return sentence

In [6]:
generator = Generator(
    model_file_path="./../Model/model.hdf5",
    vocab_file_path="./../Data/Dataset/Matthieu_processed.txt"
    )

In [10]:
generated_text = generator.generate_sentence(
                                        seed="The seed of the text to generate ...", 
                                        numbeOfCharToGenerate=100, 
                                        save_to_file=True, 
                                        file_to_save="./result.txt"
                                    )

In [11]:
print (generated_text)

the seed of the text to generate ...p à lou ; allra asssi jes huuritses  il sera donnéa unr une couierie me pripière pas de dieu vienses
